In [3]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

In [5]:
def get_source(url):
    """Return the source code of the provided url.
    
    Args: 
        url (string): URL of the page to scrape.
        
    Returns:
        response (object): HTTP response object from requests_html
    """  
    
    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    
    except requests.exceptions.RequestException as e:
        print(e)

In [9]:
def scrape_google(query):
    
    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)
    response1 = get_source("https://www.google.com/search?q=" + query + '&start=10')
    
    links = list(response.html.absolute_links)
    links += list(response1.html.absolute_links)
    google_domains = ('https://www.google.',
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')
    
    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)
    
    return links

In [10]:
scrape_google("make worklife healthier")

['https://www.naukri.com/blog/16-ways-to-maintain-work-life-balance/',
 'https://www.businessnewsdaily.com/5244-improve-work-life-balance-today.html',
 'https://www.betterup.com/blog/how-to-have-good-work-life-balance',
 'https://www.themuse.com/advice/37-tips-for-a-better-worklife-balance',
 'https://thehappinessindex.com/blog/importance-work-life-balance',
 'https://blog.vantagecircle.com/work-life-balance/',
 'https://www.betterup.com/blog/work-life-balance',
 'https://www.forbes.com/sites/deborahlee/2014/10/20/6-tips-for-better-work-life-balance/',
 'https://www.healthline.com/health/mental-health/work-life-balance',
 'https://health.clevelandclinic.org/work-life-balance/',
 'https://novoresume.com/career-blog/work-life-balance',
 'https://www.seek.com.au/career-advice/article/5-steps-to-start-improving-your-work-life-balance',
 'https://www.thepalmergroup.com/resources/blog/6-easy-ways-to-improve-your-work-life-balance',
 'https://www.zenefits.com/workest/work-life-balance-tips/',